# 크롤러 종류
### beautifulsoup4
- 서버 응답으로 정보를 추출할 수 있을 때

### selenium
- 클라이언트에서 추가 작업이 필요할 경우

# 패키지 설치

In [1]:
!pip install pandas
!pip install openpyxl

In [2]:
!pip install selenium

# 크롤링 URL 목록 생성

- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [3]:
import pandas as pd

In [4]:
# 데이터 ... 으로 자르지 않고 다 보기
pd.set_option('display.max_colwidth', None)

In [5]:
url_list_file = './weburl_list.csv'

url_list_df = pd.read_csv(url_list_file)

In [6]:
url_list_df

,가계명,placeid
0,사위식당 정자본점,1017221386
1,진대감 분당점,1567653802


In [7]:
placeid_list = url_list_df['placeid']
print(placeid_list)

0    1017221386
1    1567653802
Name: placeid, dtype: int64


In [8]:
url_list = placeid_list.map('https://m.place.naver.com/restaurant/{}/review/visitor'.format)
url_list.head()

0    https://m.place.naver.com/restaurant/1017221386/review/visitor
1    https://m.place.naver.com/restaurant/1567653802/review/visitor
Name: placeid, dtype: object

#  웹사이트 데이터 가지고 오기

In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

- https://sites.google.com/a/chromium.org/chromedriver/home

In [10]:
webdriver_options = webdriver.ChromeOptions()

# headless 모드의 경우 브라우저가 열리지 않고, 내부적으로 작동
# webdriver_options.add_argument('headless')

chromedriver = './chromedriver'
driver = webdriver.Chrome(chromedriver, options=webdriver_options)

### selector 추출법
- chrome
- 추출할 요소에 마우스 올리고 (우클릭 > 검사)
- 우클릭 메뉴 > copy > copy selector 

In [11]:
review_count_selector = '#app-root > div > div > div.place_detail_wrapper > div:nth-child(5) > div:nth-child(4) > div:nth-child(5) > h2 > span'

In [15]:
page_open_max_delay_wait_sec = 10

# 그냥 임의로 대기하는 방법
# driver.implicitly_wait(10)

def get_review_count(url):
    driver.get(url)

    # 페이지 랜더링이 끝나고 특정 element를 접근 할 수 있을 떄까지 기다림
    until_condition = expected_conditions.presence_of_element_located(
        (By.CSS_SELECTOR, review_count_selector)
    )
    elements = WebDriverWait(driver, page_open_max_delay_wait_sec).until(until_condition)
    return elements.get_attribute('innerHTML')
    
result_list = url_list.map(get_review_count)

In [16]:
result_list

0    769
1    166
Name: placeid, dtype: object

In [17]:
result_df = url_list_df.copy()
result_df['review_count'] = result_list

In [18]:
result_df

,가계명,placeid,review_count
0,사위식당 정자본점,1017221386,769
1,진대감 분당점,1567653802,166


In [19]:
result_file_name = 'review_count.xlsx'

In [20]:
result_df.to_excel(result_file_name)

# close browser

In [21]:
driver.quit()